# Downsample OpenEphys data

Import packages

In [ ]:
import quantities as pq
import numpy as np
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import signal
import os
from IPython.display import display
from ipyfilechooser import FileChooser
import json 
%matplotlib widget

Choose recording

In [ ]:
try: # tries to retrieve dpath either from a previous run or from a previous notebook
    %store -r dpath
    #dpath=Path(dpath).parent
except:
    print("the path was not defined in store")
    #dpath = "/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/ExampleRedLines/2022_08_06/13_30_01/My_V4_Miniscope/"
    dpath = "//10.69.168.1/crnldata/forgetting/Aurelie/"


fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Go inside the folder containing the LFP raw file</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

Load recordings

In [ ]:
folder_base = Path(dpath) 
LFPfile=Path(f'{folder_base}\continuous.dat')

if LFPfile.exists():
    # Load LFPs data 
    print('.dat file')
    DataRec = np.fromfile(LFPfile, dtype="int16")
    filepath_metadata = Path(os.path.join(folder_base, f'structure.oebin'))
    with open(filepath_metadata) as f:
        metadata = json.load(f)
    samplerate=metadata['continuous'][0]['sample_rate']  
    numchannel=metadata['continuous'][0]['num_channels']
    All = DataRec.reshape(-1,numchannel)

    # Load LFPs timestamps 
    for file_pathTS in folder_base.glob('**/continuous/*/timeStamps.npy'):
        print('LFPs timestamps file = ', file_pathTS)
        LFPtimestamps = np.load(file_pathTS)  

print('sample rate =', samplerate, 'Hz')
print(numchannel, 'channels recorded')
print(round(All.shape[0]/samplerate/60), 'min of recording')

Downsample signals to 1 kz & save


In [ ]:
if samplerate > 1000:
    datalen = np.shape(All)[0]
    new_sampling_rate = 1000 # Hz
    Nmber_points = int(datalen * new_sampling_rate / samplerate)
    AllDS=signal.resample(All, Nmber_points, axis = 0)
    LFPtimestampsDS=LFPtimestamps[::10][:-1]
    samplerate=new_sampling_rate
    np.save(f'{LFPfile.parent}/continuousDS.npy', AllDS)
    np.save(f'{file_pathTS.parent}/timeStampsDS.npy', LFPtimestampsDS)
# eventually delete original files to gain space